In [1]:
clean_up=True # removes gams-related files in work-folder if true
%run StdPackages.ipynb
os.chdir(directory['py'])
import techdata_to_tree, sys, ShockFunction
import abatement_EOP_new as abatement
from DB2Gams import OrdSet as OS
os.chdir(directory['curr'])

*Specify settings:*

In [2]:
name = 'A3'
catalog = "main_catalog"
f = {'tech': directory['data']+"/techdata_" + catalog + ".xlsx",
     'other': directory['data']+"/Othdata_" + catalog + ".xlsx",
     'IOdata':directory['data'] + "/IOdata_" + catalog + ".xlsx"}
s = ['inputdisp','endofpipe','inputprices']
m = ['ID','EOP'] 
e = {'M': 'vars','M_sets': 'sets'} # dictionary used to load emissions data
other_trees = {'ID_Y': {'s': 'Y', 'read_data': {'Y': 'vars'}, 'Q2P': 'Q2P'}} # the key = tree name, s = sheet that specifies tree structure, read_data (if not None) specifies how to load data.

In [3]:
type_f = {'ID_EC': 'CES_norm',
          'ID_CU': 'MNL',
          'ID_TU': 'CET_norm',
          'ID_TX': 'CES',
          'ID_BU': 'linear_out',
          'ID_BX': 'CES',
          'EOP_CU': 'MNL',
          'EOP_TU': 'CET_norm',
          'EOP_TX': 'CES'}

*Set up nesting tree:*

In [4]:
%run ATree.ipynb

## 1: Initialize model

*Set up gams model from tree:*

In [5]:
gm = abatement.abate(nts=nts,tech=tech,use_EOP=False,work_folder=directory['work'],**{'data_folder':directory['gams'],'name': name})
gm.model.settings.catalog = catalog

*Add a couple of extra aliases, and update sets with elements:*

In [6]:
gm.add_aliases([('n','nnnn'),('n','nnnnn')])
gm.model.database.update_all_sets(clean_up=False)
# gm.model.functions = {"std_pdf":"$FUNCTION std_pdf({x}): ((1/(sqrt(2*Pi)))*exp(-(Sqr({x}))/2)) $ENDFUNCTION"}


*Initialise variables at leontief levels, and fill out with simple standard values in potentially missing domains:*

In [7]:
with open(os.getcwd()+'\\conopt_ExtraPivot.opt',"r") as file:
    kwargs_mi = {'opt_file': file.read()}

In [8]:
gm.initialize_variables_leontief()
gm.initialize_variables(**{'check_variables': True})

In [9]:
if catalog == "categoryexperiment_ID":
    gm.setstate("ID")

## 2: Solve model


*Set up baseline model and solve:*

In [10]:
gm.write_and_run(name='baseline', kwargs_mi=kwargs_mi)
db0 = gm.model_instances['baseline'].out_db


*Read back solution to main db:*

In [11]:
gm.model.database.merge_dbs(gm.model.database,db0,'second')

*Turn exogenous parameters into place*

In [12]:
db_star = DataBase.GPM_database(**{'name': 'db_star'})
sigma_star = gm.get("sigma").copy()
eta_star = gm.get("eta").copy()
mu_star = gm.get("mu").copy()
if "EOP" in gm.state:
    sigmaG_star = gm.get("sigmaG").copy()

In [13]:
eta_star[:] = -5
# sigma_star[gm.get("kno_ID_EC")] = 0.2
sigma_star[gm.get("kno_ID_CU")] = 10
sigma_star[gm.get("kno_ID_BX")] = 0.5
# sigma_star["Y"] = 0.5

mu_star[gm.get("map_ID_CU")] = 1.0

if "EOP" in gm.state:
    mu_star[gm.get("map_EOP_CU")] = 1.0
    sigma_star[gm.get("kno_EOP_CU")] = 10
    sigmaG_star[:] = 0.1
    db_star["sigmaG"] = sigmaG_star
    
db_star["sigma"] = sigma_star
db_star["eta"] = eta_star
db_star["mu"] = mu_star

In [14]:
name='v0'
gm.write_and_run(name=name, add_checkpoint=name, kwargs_mi=kwargs_mi)
gm.checkpoints['v1'] = gm.model_instances[name].ws.add_checkpoint()
print(gm.model_instances[name].solve_sneakily(db_star=db_star,from_cp = True, cp_init = gm.checkpoints[name], options_run={'checkpoint': gm.checkpoints['v1']}, kwargs_shock={"n_steps":100}))
db1 = gm.model_instances[name].out_db
gm.model.database.merge_dbs(gm.model.database,db1,'second')

{'Modelstat': 16.0, 'Solvestat': 1.0}


*Change to calibration state and resolve:*

In [15]:
state = "ID_calibrate_CNS"

In [16]:
gm.reset_settings()
gm.setstate(state,init=False)

In [17]:
gm.var_exo("qsumX")

n   nn         
EH  K              14.022949
    electricity    14.022945
    gas            10.240908
    oil            12.542501
Name: qsumX, dtype: float64

In [18]:
gm.var_exo("qD")

n
C0_EH          29.000001
C_EH_1          6.000001
C_EH_2          9.000001
C_EH_3          6.000001
EH             50.000001
K              28.045889
electricity    28.045884
gas            20.481815
oil            25.085000
Name: qD, dtype: float64

In [19]:
gm.var_endo("mu")

n                        nn         
C0_EH                    EH             0.58
C_EH_1                   EH             0.12
C_EH_2                   EH             0.18
C_EH_3                   EH             0.12
EH                       out1           0.50
Y_K                      Y              0.25
Y_electricity            Y              0.25
Y_gas                    Y              0.25
Y_oil                    Y              0.25
basetech_EH_K            basetech_EH    0.25
basetech_EH_electricity  basetech_EH    0.25
basetech_EH_gas          basetech_EH    0.25
basetech_EH_oil          basetech_EH    0.25
Name: mu, dtype: float64

In [20]:
gm.write_and_run(name=state,kwargs_init={'check_variables':True},overwrite=True, kwargs_mi=kwargs_mi)
db0 = gm.model_instances[state].out_db

In [19]:
gm.model.database.merge_dbs(gm.model.database,db0,"second")

In [20]:
gm.export(name=catalog)

'C:\\Users\\zgr679\\Documents\\GitHub\\GPM_v05\\examples\\Abatement\\gamsmodels\\main_catalog'

### Make this work:

In [21]:
gm.model_instances[state].modelstat

2.0

In [22]:
db = gm.add_calib_data(f["IOdata"], currapp=False)

In [23]:
gm.write_and_run(name=state+'IO',kwargs_init={'check_variables':True},overwrite=True,add_checkpoint=state+'IO', kwargs_mi=kwargs_mi)
gm.checkpoints[state+'IO' + "_cp"] = gm.model_instances[state+'IO'].ws.add_checkpoint()

In [24]:
store_sol = {var:{"domains":"l1"} for var in ["qD", "PwThat", "mu", "gamma_tau"]}

In [25]:
print(gm.model_instances[state+'IO'].solve_sneakily(db_star=db, from_cp = True, cp_init = gm.checkpoints[state+'IO'], options_run={'checkpoint': gm.checkpoints[state+'IO'+"_cp"]}, kwargs_shock={"n_steps":100, "store_sol":store_sol, 'solvetext': gm.model.settings.get_conf('solve')}))

GamsExceptionExecution: GAMS return code not 0 (3), check C:\Users\zgr679\Documents\GitHub\GPM_v05\work_folder\_gams_py_gjo5.lst for more details